In [81]:
import pandas as pd
import json
import datetime as dt

In [82]:
with open('../DataScraping/ticker_performance.json', 'r') as json_file:
    datas = json.load(json_file)

In [83]:
for data in datas:
    del data["queryCount"]
    del data["resultsCount"]
    del data["adjusted"]
    del data["status"]
    del data["request_id"]
    del data["count"]

    for day in data['results']:
        day['ticker'] = data['ticker']

    del data["ticker"]

with open('../DataScraping/ticker_performance_changed.json', 'w') as new_file:
    json.dump(datas, new_file)


In [84]:
with open('../DataScraping/ticker_performance_changed.json', 'r') as json_file:
    datas = json.load(json_file)

output_list = []
for data in datas:
    for x in data['results']:
        output_list.append(x)


len(output_list)

125265

In [85]:
df = pd.DataFrame(output_list)
df.tail()

,v,vw,o,c,h,l,t,n,ticker
125260,2087429.0,155.4675,155.63,155.33,156.6700,154.58,1670216400000,32138,ZTS
125261,1964771.0,153.4947,154.22,153.05,155.5000,152.09,1670302800000,35035,ZTS
125262,2444170.0,150.5471,152.96,150.25,153.7892,149.38,1670389200000,38993,ZTS
125263,2267517.0,153.0717,150.53,153.68,154.3500,149.20,1670475600000,39624,ZTS
125264,3274861.0,153.6226,153.94,153.39,156.3300,152.74,1670562000000,42309,ZTS


In [86]:
df = df[['ticker','t','n','v','vw','o','c','h','l']].rename(columns={'v':'Volume',\
                                                            'vw':'VWAP',\
                                                            'o':'Open', 'c':'Close',\
                                                            'h':'High', 'l':'Low',\
                                                            't':'Date','n':'TradeCount',\
                                                            'ticker':'Ticker'})

df['Date'] = pd.to_datetime(df['Date'], origin='unix', unit='ms').dt.date

In [87]:
df.head()

,Ticker,Date,TradeCount,Volume,VWAP,Open,Close,High,Low
0,MMM,2021-12-13,36614,2530053.0,174.9704,177.00,174.58,177.3681,174.410
1,MMM,2021-12-14,36169,2327593.0,175.2165,174.85,174.72,176.6000,174.250
2,MMM,2021-12-15,31755,2139874.0,175.3975,175.08,175.78,176.2200,173.880
3,MMM,2021-12-16,33618,2249979.0,177.9551,176.71,178.31,178.9400,174.815
4,MMM,2021-12-17,41628,4012672.0,175.7735,177.86,174.75,178.3200,174.340


In [90]:
df['Year'] = ""
df['Week'] = ""
for index, value in df.iterrows():
    df['Year'][index] = df['Date'][index].isocalendar()[0]
    df['Week'][index] = df['Date'][index].isocalendar()[1]

### lamda functions don't work for .isocalendar() :(
# df2 = df.assign(Week = lambda x: (x['High'] * 1000))
# df2.head()

c:\Users\e1317395\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\e1317395\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [91]:
df.set_index('Ticker', inplace=True)

In [92]:
weekly_df = df.groupby(['Ticker','Year','Week']).agg({'TradeCount':'sum',\
    'Volume':'sum','VWAP':'mean','High':'max','Low':'min',\
        'Open':'first','Close':'last'})
        
weekly_df.head(54)

TradeCount       Volume        VWAP      High       Low  \
Ticker Year Week                                                            
A      2021 50        126074   10052235.0  152.804000  157.9000  148.0400   
            51         90740    6842750.0  154.399400  158.8600  148.5400   
            52         73745    4635664.0  159.848280  162.6200  157.1850   
       2022 1         149498   10567745.0  150.433320  159.4400  145.0900   
            2         139146   10960406.0  145.641140  150.3900  140.8600   
            3         104371    7119621.0  140.552500  143.6000  137.4900   
            4         132292    9750811.0  134.577920  138.4900  131.2150   
            5         103155    6836327.0  141.225880  144.7600  136.9729   
            6          91802    6562058.0  141.171560  145.1000  136.2400   
            7         183698   13952999.0  133.786840  138.0600  129.7100   
            8         179808   13568213.0  129.570525  134.3700  123.0600   
            9         197757   15034255.0  133.268600  138.0000  129.1550   
            10        161679   10306468.0  131.742500  136.4400  127.5400   
            11        138644    8993959.0  133.532500  139.3930  126.9000   
            12        117028    7350680.0  136.448760  140.1997  133.4482   
            13        140111    9129848.0  135.351940  139.3200  131.8600   
            14        123833    9461845.0  134.344300  137.0800  129.8600   
            15        136459   11062294.0  128.209475  133.8788  124.9100   
            16        139055    9328265.0  125.802880  131.0300  121.1500   
            17        170985   12597637.0  119.518980  123.8900  117.2400   
            18        174966   11187026.0  121.833880  126.6900  116.4900   
            19        146876    8276904.0  116.058040  120.7150  112.6400   
            20        154106   10468624.0  122.052060  125.5800  118.8300   
            21        187099   12216336.0  124.950660  130.7700  119.1800   
            22        125418    8327875.0  126.917000  130.0700  122.3000   
            23        112567    5790357.0  126.050560  131.2000  119.9000   
            24        158559    8889196.0  115.142920  117.7700  112.6700   
            25        119078    9755477.0  116.403250  121.3500  112.5200   
            26        115839    6870176.0  118.784280  122.2950  116.0000   
            27         90165    5615819.0  121.412575  123.2200  116.4900   
            28        123814    6195979.0  118.993320  123.4600  113.0200   
            29        116036    6553747.0  121.297840  128.5399  114.4600   
            30        116055    7776218.0  128.024240  134.5000  123.1900   
            31        100495    6165107.0  133.529900  135.2600  131.0900   
            32        101780    5202221.0  131.988800  135.1300  128.0200   
            33        158484   10429013.0  137.619100  149.0000  131.6600   
            34        106474    5681352.0  133.447940  136.0800  129.7800   
            35        127007    7567261.0  128.231060  130.2700  126.2100   
            36         99567    6568540.0  132.867925  137.9200  127.7200   
            37        135615    8762201.0  134.330540  140.0000  128.2800   
            38        116276    6300414.0  126.690720  130.7900  121.4400   
            39        115345    7071786.0  122.675820  125.1250  120.6000   
            40        115848    6931768.0  129.801000  133.8200  121.6800   
            41        125656    7071751.0  125.976220  130.2200  122.3500   
            42        105860    6234459.0  129.361740  134.6750  124.8200   
            43         99849    5465905.0  135.688280  139.3100  129.8050   
            44        114211    6768168.0  137.262100  141.3500  130.6600   
            45        126237    6032476.0  141.672820  149.7400  135.5600   
            46        127883    6731447.0  146.606720  149.7100  142.1600   
            47        124615    7291425.0  153.270000  157.4400  144.4300   
            48

In [116]:
json_df = weekly_df.reset_index()

,Ticker,Year,Week,TradeCount,Volume,VWAP,High,Low,Open,Close
0,A,2021,50,126074,10052235.0,152.80400,157.90,148.040,157.34,151.42
1,A,2021,51,90740,6842750.0,154.39940,158.86,148.540,150.76,157.80
2,A,2021,52,73745,4635664.0,159.84828,162.62,157.185,158.82,159.65
3,A,2022,1,149498,10567745.0,150.43332,159.44,145.090,159.00,145.15
4,A,2022,2,139146,10960406.0,145.64114,150.39,140.860,143.29,144.68


In [122]:
json_df.to_json('../CleanedData/ticker_performance_weekly.json', index=False, orient='table')

In [126]:
with open('../CleanedData/ticker_performance_weekly.json', 'r') as json_file:
    datas = json.load(json_file)

data = datas['data']

In [127]:
with open('../CleanedData/ticker_performance_weekly_cleaned.json', 'w') as new_file:
    json.dump(data, new_file)